<a href="https://colab.research.google.com/github/InowaR/colab/blob/main/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
content = requests.get("https://raw.githubusercontent.com/amephraim/nlp/master/texts/J.%20K.%20Rowling%20-%20Harry%20Potter%201%20-%20Sorcerer's%20Stone.txt").text
open("harry-potter.txt", "w", encoding="utf-8").write(content)

439742

In [2]:
import numpy as np
import sys
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.optimizers import RMSprop


with open('harry-potter.txt', 'r') as data:
  text = data.read()

# Создаем словарь уникальных символов
chars = sorted(list(set(text)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# Генерируем обучающие данные и метки
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
X = np.zeros((len(sentences), maxlen, len(chars)))
y = np.zeros((len(sentences), len(chars)))
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

optimizer = RMSprop(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)
model.fit(X, y, batch_size=128, epochs=3)

model.save('text_model.keras')

Epoch 1/3
1146/1146 [==============================] - 159s 137ms/step - loss: 2.0858
Epoch 2/3
1146/1146 [==============================] - 151s 132ms/step - loss: 1.6478
Epoch 3/3
1146/1146 [==============================] - 156s 136ms/step - loss: 1.5326


In [3]:
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

model.load_weights('text_model.keras')

# Генерируем текст с помощью обученной модели
start_index = np.random.randint(0, len(text) - maxlen - 1)
generated_text = text[start_index: start_index + maxlen]
for temperature in [0.2, 0.5, 1.0, 1.2]:
    print('----- temperature:', temperature)
    sys.stdout.write(generated_text)
    for i in range(100):
        sampled = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(generated_text):
            sampled[0, t, char_indices[char]] = 1.
        preds = model.predict(sampled, verbose=0)[0]
        next_index = np.random.choice(len(chars), p=preds)
        next_char = indices_char[next_index]
        generated_text += next_char
        generated_text = generated_text[1:]
        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()

----- temperature: 0.2
 few things we wanted to ask you, as a marbly by invicomers the opbootts. ' See, on the might
 HoSw?"

"Krestrilind himself To
persest conce
----- temperature: 0.5
"

"Krestrilind himself To
persest concected and
Derie it Hagord. Harry -- and bless,  none of Hogwarts. Re, in something burs of -- yeact t
----- temperature: 1.0
rts. Re, in something burs of -- yeact then everyound him. An
this wing. saCk.

"I know I coure of, Harry's can andrestod on the
gilnon
Lo
g
----- temperature: 1.2
Harry's can andrestod on the
gilnon
Lo
go?"

"Fentwees and Fred hheck back herrectica, -- his liss areiaser to go an warcherly be outside on
